<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/banner_IA.png"  width="1000px" height="200px">

# **Taller 03:  Pandas**

## **Outline**

1. [Ejercicio 1.](#eje1)
2. [Ejercicio 2.](#eje2)
3. [Ejercicio 3.](#eje3)
4. [Ejercicio 4.](#eje4)
5. [Ejercicio 5.](#eje5)



In [1]:
#@title **Execute this cell**
#@markdown Please include your student id
import sys
import inspect

group_id = "IA1-20252-G1" #@param {type:"string"}
assignment_id = group_id +'.taller_pandas'
student_id = "2204002" #@param {type:"string"}
"""
Put your student ID here

Example: student_id =  '2152145'
"""

"\nPut your student ID here\n\nExample: student_id =  '2152145'\n"

In [2]:
 #@title **Execute this cell**
#@markdown **UTILS**
#@markdown Please dont modify any line in this cell

import os
import json
import requests
from collections import namedtuple


Config = namedtuple('Config', ['server_name'])
config = Config(server_name='https://bivlabgrader.azurewebsites.net/api')


def check_solution_and_evaluate(assignment_id: str, student_func_str: str):

    # Set the endpoint and payload.
    payload = {
        'func_str': student_func_str,
        'assignment_id': assignment_id,
        'student_id': student_id
    }
    endpoint_url = config.server_name + '/CheckAndEvaluateSolution'
    # print(endpoint_url)

    # Make request to server with the data coming from the notebook.
    r = requests.post(endpoint_url, params=payload)
    pprint_json_response(r.json())
    return r


def pprint_json_response(response, indent=0):
    """Pretty print the response."""
    for key, value in response.items():
        print('\t' * indent + str(key.capitalize()))

        # If dictionary, do a recurrent call.
        if isinstance(value, dict):
            pprint_json_response(value, indent + 1)
        else:
            # Enumerate elements if list.
            if isinstance(value, list):
                if len(value) == 1:
                    print('\t' * (indent + 1) + str(value[0]))
                else:
                    for i, e in enumerate(value, start=1):
                        print('\t' * (indent + 1) + f'{i}. {e}')
            else:
                print('\t' * (indent + 1) + str(value))

In [3]:
#@title **Import libraries**

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(21)

---
# **Ejercicio 1**  <a name="eje1"></a>
---

#  Imputación condicional de valores faltantes

## Contexto
En un DataFrame con dos columnas:

* `value`: variable numérica que puede contener valores faltantes (`NaN`).
* `is_normal`: columna booleana que indica la estrategia de imputación para **esa fila**.

Debes reemplazar los `NaN` de `value` según:

* Si `is_normal == True` → imputar con la **media** de `value` (ignorando `NaN`).
* Si `is_normal == False` → imputar con la **mediana** de `value` (ignorando `NaN`).

Los valores no faltantes deben permanecer iguales.
Si **toda** la columna `value` es `NaN`, tanto la media como la mediana serán `NaN` y, por lo tanto, los faltantes permanecerán como `NaN` (comportamiento esperado).<br><br>

## Tu tarea
Desarrollar la función `impute_conditional(df)` la cual recibe un dataframe `(df)`, de manera que se cumplan los requerimientos solicitados.<br><br>

## Prueba tu solución
Se entregan seis (6) posibles casos de uso para poner a prueba su solución.



In [4]:
#@title **code student**
def impute_conditional(df):
    import pandas as pd

    # Calcular media y mediana (ignorando NaN)
    mean_val = df["value"].mean(skipna=True)
    median_val = df["value"].median(skipna=True)

    # Copia del DataFrame para no modificar el original
    out = df.copy()

    # Imputación condicional
    out.loc[out["value"].isna() & (out["is_normal"] == True), "value"] = mean_val
    out.loc[out["value"].isna() & (out["is_normal"] == False), "value"] = median_val

    return out

In [5]:
#@title **check your answer**

import pandas as pd

# ---------------------------
# Caso 1: Ejemplo simple
# ---------------------------
df1 = pd.DataFrame({
    "value": [10.0, None, 20.0, None, 30.0],
    "is_normal": [True, True, False, False, True]
})
# media = (10 + 20 + 30)/3 = 20.0 ; mediana = 20.0
expected1 = pd.DataFrame({
    "value": [10.0, 20.0, 20.0, 20.0, 30.0],
    "is_normal": [True, True, False, False, True]
})
print(impute_conditional(df1).equals(expected1))  # ✅ True si es correcto


# ---------------------------
# Caso 2: Típico con media ≠ mediana
# ---------------------------
df2 = pd.DataFrame({
    "value": [5.0, None, 100.0, None, 7.0, 9.0, None],
    "is_normal": [True, False, True, False, True, True, False]
})
# valores no-NaN: [5, 100, 7, 9] -> media = 30.25 ; mediana = (7 + 9)/2 = 8.0
expected2 = pd.DataFrame({
    "value": [
        5.0,     # se queda
        8.0,     # NaN & is_normal=False -> mediana 8.0
        100.0,   # se queda
        8.0,     # NaN & is_normal=False -> mediana 8.0
        7.0,     # se queda
        9.0,     # se queda
        8.0      # NaN & is_normal=False -> mediana 8.0
    ],
    "is_normal": [True, False, True, False, True, True, False]
})
print(impute_conditional(df2).equals(expected2))  # ✅ True si es correcto


# ---------------------------
# Caso 3: Solo imputación por media (todas las filas True)
# ---------------------------
df3 = pd.DataFrame({
    "value": [None, 2.0, None, 8.0],
    "is_normal": [True, True, True, True]
})
# media = (2 + 8)/2 = 5.0 ; mediana no se usa
expected3 = pd.DataFrame({
    "value": [5.0, 2.0, 5.0, 8.0],
    "is_normal": [True, True, True, True]
})
print(impute_conditional(df3).equals(expected3))  # ✅ True si es correcto


# ---------------------------
# Caso 4: Solo imputación por mediana (todas las filas False)
# ---------------------------
df4 = pd.DataFrame({
    "value": [1.0, None, 9.0, None, 3.0],
    "is_normal": [False, False, False, False, False]
})
# valores no-NaN: [1, 9, 3] -> mediana = 3.0
expected4 = pd.DataFrame({
    "value": [1.0, 3.0, 9.0, 3.0, 3.0],
    "is_normal": [False, False, False, False, False]
})
print(impute_conditional(df4).equals(expected4))  # ✅ True si es correcto


# ---------------------------
# Caso 5: Borde - toda la columna value es NaN
# ---------------------------
df5 = pd.DataFrame({
    "value": [None, None],
    "is_normal": [True, False]
})
# media = NaN ; mediana = NaN -> permanecen NaN
expected5 = pd.DataFrame({
    "value": [None, None],
    "is_normal": [True, False]
})
print(impute_conditional(df5).equals(expected5))  # ✅ True si es correcto


# ---------------------------
# Caso 6: Sin NaN (no debe cambiar nada)
# ---------------------------
df6 = pd.DataFrame({
    "value": [2.0, 4.0, 6.0],
    "is_normal": [True, False, True]
})
expected6 = df6.copy()
print(impute_conditional(df6).equals(expected6))  # ✅ True si es correcto

True
True
True
True
True
True


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


## ✅ Salidas esperadas

```
True
True
True
True
True
True
/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
```

In [6]:
#@title **send your answer**
student_func_str = inspect.getsource(impute_conditional)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, you got the highest score.


<br><br><br>

---
# **Ejercicio 2**  <a name="eje2"></a>
---

# Nota final con penalización por entrega tardía

## Contexto

Trabajarás con un **DataFrame de notas** por asignación y estudiante. Cada fila representa la calificación de un estudiante en una actividad, el **peso** que dicha actividad aporta a la nota final y si la **entrega fue a tiempo**.
En este entorno, las entregas **fuera de tiempo** tienen una **penalización del 40%** sobre la nota de esa actividad. Además, los pesos pueden estar expresados como **proporción** (`0..1`) o como **porcentaje** (`0..100`) e incluso venir **mezclados** en la misma columna, por lo que es necesario **normalizarlos** a proporción antes de ponderar.

Columnas fijas del DataFrame:

* `student_id`: código del estudiante
* `assignment`: nombre de la asignación
* `final_weight`: peso final de la asignación (proporción `0..1` o porcentaje `0..100`)
* `grade`: nota obtenida (0–100)
* `on_time`: `True` si entregó a tiempo, `False` si fue tarde

## Tu Tarea

Implementa **`compute_final_grades(df)`** que devuelva un DataFrame **`summary_df`** con la **nota final por estudiante**, con columnas:

   * `student_id`
   * `final_grade = Σ weighted_score` de todas sus filas

## Ejemplos

### Ejemplo 1: Pesos en porcentaje y una entrega tarde

**Entrada (fragmento):**

```
student_id | assignment | final_weight | grade | on_time
-----------|------------|--------------|-------|---------
A          | T1         | 40           | 80    | True
A          | T2         | 60           | 90    | False
```

**Cálculo:**

* `weight_prop`: 40%→0.4, 60%→0.6
* `adjusted_grade`: T1=80; T2=90\*0.60=54
* `weighted_score`: T1=80 * 0.4=32; T2=54 * 0.6=32.4
* **final\_grade(A) = 32 + 32.4 = 64.4**

### Ejemplo 2: Mezcla de porcentaje y proporción, todo a tiempo

**Entrada (fragmento):**

```
student_id | assignment | final_weight | grade | on_time
-----------|------------|--------------|-------|---------
B          | P1         | 0.5          | 70    | True
B          | P2         | 50           | 100   | True
```

**Cálculo:**

* `weight_prop`: 0.5→0.5, 50%→0.5
* `adjusted_grade`: P1=70; P2=100
* `weighted_score`: P1=70 * 0.5=35; P2=100 * 0.5=50
* **final\_grade(B) = 35 + 50 = 85**


In [7]:
#@title **code student**
def compute_final_grades(df):
    import pandas as pd

    # Copiar el DataFrame para no alterar el original
    out = df.copy()

    # Normalizar pesos: porcentaje (ej: 40, 60) → proporción (0.4, 0.6)
    out["weight_prop"] = out["final_weight"].apply(
        lambda w: w / 100 if w > 1 else w
    )

    # Penalizar notas por entregas tarde
    out["adjusted_grade"] = out.apply(
        lambda row: row["grade"] if row["on_time"] else row["grade"] * 0.6,
        axis=1
    )

    # Calcular puntaje ponderado
    out["weighted_score"] = out["adjusted_grade"] * out["weight_prop"]

    # Calcular nota final por estudiante
    summary_df = (
        out.groupby("student_id", as_index=False)["weighted_score"]
        .sum()
        .rename(columns={"weighted_score": "final_grade"})
    )

    return summary_df

In [8]:
#@title **check your answer**

import pandas as pd

def df_equal_float(df1, df2):
    """Convierte todas las columnas numéricas a float antes de comparar con .equals"""
    df1c = df1.copy()
    df2c = df2.copy()
    for c in df1c.columns.intersection(df1c.select_dtypes(include="number").columns):
        df1c[c] = df1c[c].astype(float)
    for c in df2c.columns.intersection(df2c.select_dtypes(include="number").columns):
        df2c[c] = df2c[c].astype(float)
    return df1c.equals(df2c)

# ---------------------------
# Caso 1
# ---------------------------
df1 = pd.DataFrame({
    "student_id": ["A", "A", "B", "B"],
    "assignment": ["T1", "T2", "T1", "T2"],
    "final_weight": [40, 60, 40, 60],
    "grade": [80, 90, 70, 100],
    "on_time": [True, True, False, True]
})
summary1_expected = pd.DataFrame({
    "student_id": ["A", "B"],
    "final_grade": [86.0, 76.8]
})
summary1 = compute_final_grades(df1)
print("Caso 1 - summary:", df_equal_float(summary1, summary1_expected))


# ---------------------------
# Caso 2
# ---------------------------
df2 = pd.DataFrame({
    "student_id": ["S1", "S1", "S2", "S2", "S2"],
    "assignment": ["A1", "A2", "A1", "A2", "A3"],
    "final_weight": [0.25, 0.75, 0.5, 0.25, 0.25],
    "grade": [100, 80, 90, 60, 50],
    "on_time": [True, False, False, False, True]
})
summary2_expected = pd.DataFrame({
    "student_id": ["S1", "S2"],
    "final_grade": [61.0, 48.5]
})
summary2 = compute_final_grades(df2)
print("Caso 2 - summary:", df_equal_float(summary2, summary2_expected))


# ---------------------------
# Caso 3
# ---------------------------
df3 = pd.DataFrame({
    "student_id": ["X", "X", "Y"],
    "assignment": ["HW1", "HW2", "HW1"],
    "final_weight": [50, 0.5, 50],
    "grade": [80, 100, 90],
    "on_time": [True, False, False]
})
summary3_expected = pd.DataFrame({
    "student_id": ["X", "Y"],
    "final_grade": [70.0, 27.0]
})
summary3 = compute_final_grades(df3)
print("Caso 3 - summary:", df_equal_float(summary3, summary3_expected))


# ---------------------------
# Caso 4
# ---------------------------
df4 = pd.DataFrame({
    "student_id": ["Z", "Z"],
    "assignment": ["Quiz", "Proyecto"],
    "final_weight": [0, 100],
    "grade": [0, 80],
    "on_time": [False, True]
})
summary4_expected = pd.DataFrame({
    "student_id": ["Z"],
    "final_grade": [80.0]
})
summary4 = compute_final_grades(df4)
print("Caso 4 - summary:", df_equal_float(summary4, summary4_expected))


# ---------------------------
# Caso 5
# ---------------------------
df5 = pd.DataFrame({
    "student_id": ["U", "U"],
    "assignment": ["P1", "P2"],
    "final_weight": [0.5, 0.5],
    "grade": [80, 70],
    "on_time": [False, False]
})
summary5_expected = pd.DataFrame({
    "student_id": ["U"],
    "final_grade": [45.0]
})
summary5 = compute_final_grades(df5)
print("Caso 5 - summary:", df_equal_float(summary5, summary5_expected))


Caso 1 - summary: True
Caso 2 - summary: True
Caso 3 - summary: True
Caso 4 - summary: True
Caso 5 - summary: True


## ✅ Salidas esperadas

```
Caso 1 - summary: True
Caso 2 - summary: True
Caso 3 - summary: True
Caso 4 - summary: True
Caso 5 - summary: True
```

In [9]:
#@title **send your answer**
student_func_str = inspect.getsource(compute_final_grades)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, you got the highest score.


<br><br><br>

---
# **Ejercicio 3**  <a name="eje3"></a>
---

## Manejo de valores NaN

## Contexto

En la práctica clínica, la escala Hoehn & Yahr (H&Y) se utiliza con frecuencia para clasificar la progresión de la enfermedad de Parkinson, con valores que van de 0 (sin síntomas) a 5 (discapacidad severa). Dentro de las diferentes tareas motoras analizadas durante la rutina clinica, tareas como el *finger tapping, el grasping y el toe tapping* son fundamentales para calcular la Modified Bradykinesia Rating Scale (MBRS), útil para un análisis cuantitativo del deterioro neurológico. Sin embargo, es común que durante la recolección de datos clínicos haya valores faltantes (NaN) en estas columnas, lo que dificulta los análisis posteriores. Para mantener la coherencia clínica, dichos valores, una estrategia puede ser completarse teniendo en cuenta el estadio H&Y de cada paciente.

## Tu tarea

Implementar la función `fill_nan`, a la cual ingresa un DataFrame `df` que contiene las siguientes columnas:
* `"H&Y_stage"` (entero de 0 a 5)
* `"finger_tapping"` (escala 0–4, puede contener NaN)
* `"grasping"` (escala 0–4, puede contener NaN)
* `"toe_tapping"` (escala 0–4, puede contener NaN)

Tu misión es completar los valores faltantes (`NaN`) en las tres columnas motoras (``finger_tapping, grasping, toe_tapping``) agrupando por ``"H&Y_stage"`` y reemplazando los NaN con el promedio de cada columna dentro del grupo correspondiente.<br><br>Posteriormente, debes <ins>retornar</ins> el resultado.

In [39]:
#@title **code student**
def fill_nan(df):
    import pandas as pd
    import numpy as np

    # columnas motoras que queremos rellenar
    motor_cols = ["finger_tapping", "grasping", "toe_tapping"]

    # para cada columna motora, rellenar NaN con la media de su grupo H&Y_stage
    for col in motor_cols:
        df[col] = df[col].fillna(
            df.groupby("H&Y_stage")[col].transform("mean")
        )

    return df

In [40]:
#@title **check your answer**
import pandas as pd
import numpy as np

data = {
      "H&Y_stage": [1, 1, 2, 2, 3, 3],
      "finger_tapping": [1, np.nan, 2, 3, np.nan, 4],
      "grasping": [2, 1, np.nan, 3, 4, np.nan],
      "toe_tapping": [1, 2, 3, np.nan, 2, np.nan]
}
df = pd.DataFrame(data)
print(fill_nan(df))

   H&Y_stage  finger_tapping  grasping  toe_tapping
0          1             1.0       2.0          1.0
1          1             1.0       1.0          2.0
2          2             2.0       3.0          3.0
3          2             3.0       3.0          3.0
4          3             4.0       4.0          2.0
5          3             4.0       4.0          2.0


## ✅ Salidas esperadas

```
   H&Y_stage  finger_tapping  grasping  toe_tapping
0          1             1.0       2.0          1.0
1          1             1.0       1.0          2.0
2          2             2.0       3.0          3.0
3          2             3.0       3.0          3.0
4          3             4.0       4.0          2.0
5          3             4.0       4.0          2.0
```

In [41]:
#@title **send your answer**
student_func_str = inspect.getsource(fill_nan)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, you got the highest score.
Status
	You have achieved your best score: 5.0


<br><br><br>

---
# **Ejercicio 4**  <a name="eje4"></a>
---

## Análisis de visitas clínicas con resample

## Contexto
Un paciente con enfermedad de Parkinson asiste a consultas de seguimiento donde se registra su puntaje en la escala MBRS (0–12). Sin embargo, las visitas no siempre ocurren con la misma frecuencia: a veces cada semana, otras veces cada varios días. Para analizar la progresión de manera uniforme, es útil reagrupar los datos en intervalos regulares (ejemplo: mensual) y calcular estadísticos en esos intervalos.

## Tu tarea

Implementar una función `clinical_resample` que recibe como parámetro un dataframe `df` que contiene puntajes MBRS en intervalos mensuales. Para ello deberás:
- reagrupar los puntajes MBRS por meses (intervalos mensuales, según la columna `fecha`) utilizando el método `resample("ME")`.
- posteriormente, calcule los valores estadísticos `"mean", "min", "max"` que permitan ver la progresión clínica del paciente.
- **retorne** el dataframe resultante.

<ins>**Nota:**</ins> usted puede calcular dichos valores estadísticos utilizando el método `agg`, el cual permite indicar qué operaciones se desean aplicar a una (o más) columnas. Por ejemplo, si desea aplicar el promedio (`"mean"`), el valor mínimo (`"min"`) y el valor máximo (`"max"`) a una columna llamada (`"micolumna"`).

In [30]:
#@title **code student**
def clinical_resample(df):
    import pandas as pd
    # reagrupar por mes (fin de mes) y calcular mean, min y max
    # usamos "M" porque el entorno de test es viejo
    df_resample = df.resample("M").agg({"MBRS_score": ["mean", "min", "max"]})
    return df_resample

In [31]:
#@title **check your answer**
import pandas as pd

data = {
    "fecha": [
        "2023-01-05", "2023-01-12", "2023-01-28",
        "2023-02-03", "2023-02-18",
        "2023-03-02", "2023-03-15", "2023-03-30",
        "2023-04-10"
    ],
    "MBRS_score": [5, 6, 4, 7, 6, 8, 9, 8, 10]
}

df = pd.DataFrame(data)
df["fecha"] = pd.to_datetime(df["fecha"])   # convertir a datetime
df = df.set_index("fecha")                  # poner fecha como índice

# llamar a tu función
print(clinical_resample(df))

           MBRS_score        
                 mean min max
fecha                        
2023-01-31   5.000000   4   6
2023-02-28   6.500000   6   7
2023-03-31   8.333333   8   9
2023-04-30  10.000000  10  10


/tmp/ipython-input-4081426157.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_resample = df.resample("M").agg({"MBRS_score": ["mean", "min", "max"]})


## ✅ Salidas esperadas

```
           MBRS_score        
                 mean min max
fecha                        
2023-01-31   5.000000   4   6
2023-02-28   6.500000   6   7
2023-03-31   8.333333   8   9
2023-04-30  10.000000  10  10
/tmp/ipython-input-1533716997.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_resample = df.resample("M").agg({"MBRS_score": ["mean", "min", "max"]})
```


In [32]:
#@title **send your answer**
student_func_str = inspect.getsource(clinical_resample)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, you got the highest score.
Status
	You have achieved your best score: 5.0


<br><br><br>

---
# **Ejercicio 5**  <a name="eje5"></a>
---

# Suavizado de síntomas motores MBRS con rolling

## Contexto:

Antes de solucionar este punto, por favor **tenga en cuenta** el contexto clinico presentado en [1]:

En evaluaciones clínicas seriadas, los puntajes de la MBRS (*Modified Bradykinesia Rating Scale*) pueden presentar fluctuaciones leves entre mediciones sucesivas. Un método común para analizar la tendencia del paciente es aplicar un promedio móvil con ventana definida.

## Tu tarea

Implementa un la función ``mbrs_rolling``, la cual toma como parámetro un dataframe ``df`` y aplica:
- una operación `mean` por el método `rolling` que abarque 3 observaciones sobre la columna (`"MBRS_score"`) y <ins>retorne</ins> un dataframe con la señal original (`"MBRS_score"`) y su versión suavizada (`"MBRS_smooth"`).
- una operación `std` por el método `rolling` que abarque 2 observaciones sobre la columna (`"HYY_score"`) y <ins>retorne</ins> **solamente** la versión suavizada.
- una operación de correlación, con 2 observaciones de longitud, **entre las columnas** (`"MBRS_score"`) y (`"HYY_score"`), y <ins>retorne</ins> el resultado.






<br><br>
<font size="1">
[1] <ins>Contexto clínico:</ins> El diagnóstico clínico de la enfermedad de Parkinson (EP) se basa en la evaluación de síntomas motores categorizando la enfermedad en escalas como Hoehn & Yahr (H&Y). La *Modified Bradykinesia Rating Scale* (MBRS) es una escala alternativa que cuantifica la lentitud en el movimiento a través de la ejecución de tareas motoras estandarizadas, como movimientos repetitivos de dedos, manos y brazos. Esta escala comprende valores entre 0 y 12, siendo los valores altos indicadores de un mayor daño cognitivo. Estas mediciones se correlacionan con el grado de daño neurológico, permitiendo análisis más sensibles y objetivos de la progresión motora de la enfermedad.<br>
La valoración de MBRS se da asi:</font>

$$
MBRS =
\begin{cases}
\text{Low}, & \text{si } 0 < score \leq 4 \\[6pt]
\text{Medium}, & \text{si } 4 < \text{score} \leq 8 \\[6pt]
\text{High}, & \text{si } 8 < \text{score} \leq 12  \\[6pt]
\end{cases}
$$

<font size="1">
donde (`score`) se obtiene por la suma de cada uno de los valores de las columnas de un dataset, donde cada una de ellas puede tomar valores entre 0 y 4. De este modo, para valores de 3, 2, 4, el MBRS asociado seria alto (valor 9 en la escala).</font>

In [36]:
#@title **code student**
def mbrs_rolling(df):
    import pandas as pd

    # 1. DataFrame con MBRS_score original y MBRS_smooth (rolling mean 3)
    df1 = df.copy()
    df1["MBRS_smooth"] = df1["MBRS_score"].rolling(window=3).mean()

    # 2. Serie con std rolling 2 sobre HYY_score
    df2 = df["HYY_score"].rolling(window=2).std()

    # 3. Serie con correlación rolling 2 entre MBRS_score y HYY_score
    # aseguramos convertir a float
    mb = df["MBRS_score"].astype(float)
    hy = df["HYY_score"].astype(float)
    df3 = mb.rolling(window=2).corr(hy)

    return df1, df2, df3

In [37]:
#@title **check your answer**
import pandas as pd
data = {"MBRS_score": [2, 3, 4, 5, 3, 2, 4], "HYY_score": [5, 5, 4, 5, 5, 3, 5]}
df = pd.DataFrame(data)

a, b, c = mbrs_rolling(df)
print(a, "\n")
print(b, "\n")
print(c)

   MBRS_score  HYY_score  MBRS_smooth
0           2          5          NaN
1           3          5          NaN
2           4          4     3.000000
3           5          5     4.000000
4           3          5     4.000000
5           2          3     3.333333
6           4          5     3.000000 

0         NaN
1    0.000000
2    0.707107
3    0.707107
4    0.000000
5    1.414214
6    1.414214
Name: HYY_score, dtype: float64 

0    NaN
1    NaN
2   -1.0
3    1.0
4    NaN
5    1.0
6    1.0
dtype: float64


## ✅ Salidas esperadas

```
   MBRS_score  HYY_score  MBRS_smooth
0           2          5          NaN
1           3          5          NaN
2           4          4     3.000000
3           5          5     4.000000
4           3          5     4.000000
5           2          3     3.333333
6           4          5     3.000000

0         NaN
1    0.000000
2    0.707107
3    0.707107
4    0.000000
5    1.414214
6    1.414214
Name: HYY_score, dtype: float64

0         NaN
1         NaN
2   -0.866025
3    0.000000
4    0.000000
5    0.755929
6    0.866025
dtype: float64
```

In [38]:
#@title **send your answer**
student_func_str = inspect.getsource(mbrs_rolling)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	3.333333333333333
Message
	Error en correlación rolling (Test 3).
Status
	You have achieved your best score: 3.333333333333333


<br><br><br>

---
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/bannerThanks.jpg" alt="Drawing" style="width:700px;"/>